In [1]:
import os

SEASONS = list(range(2020,2025))

DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

In [3]:
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [5]:
async def get_html(url, selector, sleep=1, retries=3):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i)
        try:
            async with async_playwright() as p:
                browser = await p.firefox.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

In [6]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    standings_pages = [f"https://www.basketball-reference.com{l['href']}" for l in links]
    
    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
        
        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)

In [9]:
!playwright install

155.3 MiB [                    ] 0% 0.0s155.3 MiB [                    ] 0% 5.1s155.3 MiB [                    ] 0% 3.6s155.3 MiB [                    ] 1% 3.2s155.3 MiB [=                   ] 2% 3.0s155.3 MiB [=                   ] 3% 3.0s155.3 MiB [=                   ] 3% 3.1s155.3 MiB [=                   ] 4% 3.0s155.3 MiB [=                   ] 4% 3.1s155.3 MiB [=                   ] 5% 3.0s155.3 MiB [=                   ] 5% 2.9s155.3 MiB [=                   ] 6% 2.9s155.3 MiB [=                   ] 6% 2.8s155.3 MiB [==                  ] 7% 2.7s155.3 MiB [==                  ] 8% 2.7s155.3 MiB [==                  ] 8% 2.6s155.3 MiB [==                  ] 9% 2.6s155.3 MiB [==                  ] 10% 2.6s155.3 MiB [==                  ] 11% 2.5s155.3 MiB [==                  ] 12% 2.5s155.3 MiB [===                 ] 12% 2.4s155.3 MiB [===                 ] 13% 2.4s155.3 MiB [===                 ] 13% 2.5s155.3 MiB [===                 ] 14% 2.4s155.3 MiB [===                 ] 

In [7]:
await scrape_season(2024)

Error: BrowserType.launch: Executable doesn't exist at /home/codespace/.cache/ms-playwright/firefox-1447/firefox/firefox
╔════════════════════════════════════════════════════════════╗
║ Looks like Playwright was just installed or updated.       ║
║ Please run the following command to download new browsers: ║
║                                                            ║
║     playwright install                                     ║
║                                                            ║
║ <3 Playwright Team                                         ║
╚════════════════════════════════════════════════════════════╝

In [13]:
standings_files = os.listdir(STANDINGS_DIR)

In [11]:
async def scrape_game(standings_file):
    with open(standings_file, 'r') as f:
        html = f.read()

    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    hrefs = [l.get('href') for l in links]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in hrefs if l and "boxscore" in l and '.html' in l]

    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, "#content")
        if not html:
            continue
        with open(save_path, "w+") as f:
            f.write(html)

In [ ]:
pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 5.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 5.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 5.2 MB/s eta 0:00:00a 0:00:01
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [14]:
import pandas as pd

files = [s for s in standings_files if str(2024) in s]
    
for f in files:
    filepath = os.path.join(STANDINGS_DIR, f)
        
    await scrape_game(filepath)

Lakers vs Nuggets, October 24, 2023 | Basketball-Reference.com
Suns vs Warriors, October 24, 2023 | Basketball-Reference.com
Rockets vs Magic, October 25, 2023 | Basketball-Reference.com
Celtics vs Knicks, October 25, 2023 | Basketball-Reference.com
Wizards vs Pacers, October 25, 2023 | Basketball-Reference.com
Hawks vs Hornets, October 25, 2023 | Basketball-Reference.com
Pistons vs Heat, October 25, 2023 | Basketball-Reference.com
Timberwolves vs Raptors, October 25, 2023 | Basketball-Reference.com
Cavaliers vs Nets, October 25, 2023 | Basketball-Reference.com
Pelicans vs Grizzlies, October 25, 2023 | Basketball-Reference.com
Thunder vs Bulls, October 25, 2023 | Basketball-Reference.com
Kings vs Jazz, October 25, 2023 | Basketball-Reference.com
Mavericks vs Spurs, October 25, 2023 | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/boxscores/202310250LAC.html
Trail Blazers vs Clippers, October 25, 2023 | Basketball-Reference.com
76ers vs Bucks, October 26,